### Name : Siddhi Kothekar
### Roll no. : 19
### Batch : A2

### Practical 3

In [ ]:
from collections import defaultdict

In [ ]:
def compute_first(grammar):
    first = defaultdict(set)

    def first_of(symbol):
        if symbol in first and first[symbol]:
            return first[symbol]

        if not symbol.isupper():
            return {symbol}

        first_set = set()
        for production in grammar[symbol]:
            for char in production:
                char_first = first_of(char)
                first_set.update(char_first - {'ε'})
                if 'ε' not in char_first:
                    break
            else:
                first_set.add('ε')

        first[symbol] = first_set
        return first_set

    for non_terminal in grammar:
        first_of(non_terminal)

    return first


In [ ]:
def parse_grammar():
    grammar = defaultdict(list)
    n = int(input("Enter number of productions: "))
    print("Enter productions (Use '|' for multiple productions, 'ε' for epsilon):")
    for _ in range(n):
        line = input().strip().replace(" ", "")
        lhs, rhs = line.split("->")
        productions = rhs.split("|")
        grammar[lhs].extend(productions)
    return grammar

In [ ]:
def main():
    grammar = parse_grammar()
    first_sets = compute_first(grammar)
    print("\nFIRST sets:")
    for non_terminal, first_set in first_sets.items():
        print(f"FIRST({non_terminal}) = {first_set}")

In [ ]:
if __name__ == "__main__":
    main()

Enter number of productions: 4
Enter productions (Use '|' for multiple productions, 'ε' for epsilon):
S->AB|C
A->a|b|ε
B->p|ε
C->c

FIRST sets:
FIRST(A) = {'ε', 'b', 'a'}
FIRST(B) = {'p', 'ε'}
FIRST(C) = {'c'}
FIRST(S) = {'b', 'a', 'p', 'c', 'ε'}


In [ ]:
def compute_follow(grammar, first_sets):
    follow = defaultdict(set)
    start_symbol = list(grammar.keys())[0]
    follow[start_symbol].add('$')

    changed = True
    while changed:
        changed = False
        for non_terminal, productions in grammar.items():
            for production in productions:
                for i, symbol in enumerate(production):
                    if symbol.isupper():
                        # Rule 2: if A -> αBβ, FOLLOW(B) contains everything in FIRST(β) - {ε}
                        if i < len(production) - 1:
                            beta = production[i+1:]
                            beta_first = set()
                            for char in beta:
                                beta_first.update(first_sets[char] if char.isupper() else {char})
                                if 'ε' not in beta_first:
                                    break

                            follow_before = len(follow[symbol])
                            follow[symbol].update(beta_first - {'ε'})
                            if len(follow[symbol]) > follow_before:
                                changed=True

                        # Rule 3: if A -> αB or A -> αBβ, where ε ∈ FIRST(β), then FOLLOW(B) contains FOLLOW(A)
                        if i == len(production) - 1 or ('ε' in beta_first if i < len(production) - 1 else True):
                          follow_before = len(follow[symbol])
                          follow[symbol].update(follow[non_terminal])
                          if len(follow[symbol]) > follow_before:
                                changed=True
    return follow



In [ ]:

def construct_parsing_table(grammar, first_sets, follow_sets):
    table = defaultdict(dict)
    for non_terminal, productions in grammar.items():
        for production in productions:
            first_prod = set()
            for symbol in production:
                first_prod.update(first_sets[symbol] if symbol.isupper() else {symbol})
                if 'ε' not in first_prod:
                    break

            if 'ε' in first_prod:
              for terminal in follow_sets[non_terminal]:
                 if table[non_terminal].get(terminal,None) != None:
                     print ("Error: Grammar is not LL(1)")
                     return None
                 table[non_terminal][terminal] = production

            else:
              for terminal in first_prod:
                if table[non_terminal].get(terminal,None) != None:
                     print ("Error: Grammar is not LL(1)")
                     return None
                table[non_terminal][terminal] = production
    return table

In [ ]:

def main():
    grammar = parse_grammar()
    first_sets = compute_first(grammar)
    print("\nFIRST sets:")
    for non_terminal, first_set in first_sets.items():
        print(f"FIRST({non_terminal}) = {first_set}")

    follow_sets = compute_follow(grammar, first_sets)
    print("\nFOLLOW sets:")
    for non_terminal, follow_set in follow_sets.items():
        print(f"FOLLOW({non_terminal}) = {follow_set}")

    parsing_table = construct_parsing_table(grammar,first_sets,follow_sets)

    if parsing_table:
        print("\nLL(1) Parsing Table:")
        for non_terminal, terminals in parsing_table.items():
          for terminal, production in terminals.items():
            print(f"M[{non_terminal}, {terminal}] = {non_terminal}->{production}")

In [ ]:

if __name__ == "__main__":
    main()

Enter number of productions: 6
Enter productions (Use '|' for multiple productions, 'ε' for epsilon):
S->aBDh
B->cC
C->bC
D->EF
E->g|ε
F->f|ε

FIRST sets:
FIRST(S) = {'a'}
FIRST(B) = {'c'}
FIRST(C) = {'b'}
FIRST(E) = {'g', 'ε'}
FIRST(F) = {'ε', 'f'}
FIRST(D) = {'g', 'ε', 'f'}

FOLLOW sets:
FOLLOW(S) = {'$'}
FOLLOW(B) = {'g', '$', 'h', 'f'}
FOLLOW(D) = {'h'}
FOLLOW(C) = {'g', '$', 'h', 'f'}
FOLLOW(E) = {'h', 'f'}
FOLLOW(F) = {'h'}

LL(1) Parsing Table:
M[S, a] = S->aBDh
M[B, c] = B->cC
M[C, b] = C->bC
M[D, h] = D->EF
M[E, g] = E->g
M[E, h] = E->ε
M[E, f] = E->ε
M[F, f] = F->f
M[F, h] = F->ε
